In [1]:
import pandas as pd
silver_df_main = pd.read_csv('data/sentence_level_evidence.csv')
silver_df_main

,SUBJECT_ID,HADM_ID,TEXT,ICD9_CODE
0,55677,108847.0,Admission Date: [**2134-4-5**] D...,"[""[('This 79 year old male who is known to car..."
1,55122,160937.0,Admission Date: [**2192-3-27**] ...,['[(\'Patient underwent uncomplicated penile i...
2,11892,193055.0,Admission Date: [**2152-6-26**] Dischar...,"[""[('Mr. [**Known lastname 36243**] is a 67-ye..."
3,13913,153057.0,Admission Date: [**2152-3-9**] Discharg...,"[""[\n ('6. Diabetes mellitus.', '250.00'),..."
4,21553,188632.0,Admission Date: [**2184-12-24**] ...,"[""[('Patient recorded as having No Known Aller..."
...,...,...,...,...
1445,70182,182356.0,Admission Date: [**2110-2-9**] D...,"[""[('75 year old female with history of mitral..."
1446,25235,182008.0,Admission Date: [**2167-6-10**] ...,"['[\n (""52 yo M with PMH DM I, CAD s/p CABG..."
1447,18736,152789.0,Admission Date: [**2109-8-30**] ...,"[""[\n ('Patient recorded as having No Known..."
1448,42757,167098.0,Admission Date: [**2154-10-6**] ...,"[""[\n ('The patient is a 59-year-old man wi..."


In [2]:
print(eval(silver_df_main.iloc[0,-1])), print(type(eval(silver_df_main.iloc[0,-1])))

["[('This 79 year old male who is known to cardiac surgery with critical aortic stenosis, having refused surgical intervention in the past, s/p valuloplasy x2 [**2132-10-15**]/[**2133-12-15**], who was transferred from an OSH for acute chest pain with troponin bump to 4.4.', '427.31'), ('He was transfered to [**Hospital1 18**] for further mangangment of aortic stenosis.', '427.31'), ('IMPRESSION: Severe aortic valve stenosis.', '427.31')]", "[('Novolog insulin 8 units [**Hospital1 **] (before breakfast and before\\nsupper)', '250.02'),\n('Insulin lispro', '250.02'),\n('metoprolol tartrate 25 mg Tablet Sig: One (1) Tablet PO BID\\n(2 times a day)', '272.0'),\n('Lasix 80 mg Tablet Sig: One (1) Tablet PO twice a day for 2\\nweeks.', '599.70'),\n('Lasix 40mg QDAY', '599.70'),\n('magnesium hydroxide 400 mg/5 mL Suspension Sig: Thirty (30)\\nML PO HS (at bedtime) as needed for constipation.', '682.9'),\n('potassium chloride 20 mEq Tablet, ER Particles/Crystals Sig:\\nOne (1) Tablet, ER Parti

(None, None)

In [3]:
# import re
# icd_regex = r'(?:\b\w+\b\s*)?\b\d{3,5}\.?\d{2,4}\b'
# code_list = []
# seen_codes = set()
# for x in eval(silver_df_main.iloc[0,-1]):
#     # code_list.append(re.findall(icd_regex, x))
#     codes = re.findall(icd_regex, x)
#     for code in codes:
#         if code not in seen_codes:
#             seen_codes.add(code)
#             code_list.append(code)
#     # print(x)

# print(code_list)

## Processing the data

In [4]:
candidate_list = """
    V70.0 - Routine Medical Exam
    V76.2 - Screening for Malignant Neoplasm of Cervix
    401.1 - Benign Hypertension
    V76.44 - Screening for Malignant Neoplasm of Prostate
    250.00 - Diabetes Mellitus without Complications, Type II or Unspecified Type
    401.9 - Hypertension, Unspecified
    780.79 - Other Malaise and Fatigue
    599.0 - Urinary Tract Infection, Site Not Specified
    272.4 - Other and Unspecified Hyperlipidemia
    V72.60 - Laboratory Examination, Unspecified
    244.9 - Hypothyroidism, Unspecified
    V58.69 - Long-term (current) Use of Other Medications
    788.1 - Dysuria
    272.2 - Mixed Hyperlipidemia
    272.0 - Pure Hypercholesterolemia
    268.9 - Unspecified Vitamin D Deficiency
    462 - Acute Pharyngitis
    250.02 - Diabetes Mellitus without Complications, Type II or Unspecified Type, Uncontrolled
    V58.61 - Long-term (current) Use of Anticoagulants
    789.00 - Abdominal Pain, Unspecified Site
    285.9 - Anemia, Unspecified
    427.31 - Atrial Fibrillation
    599.70 - Hematuria, Unspecified
    787.91 - Diarrhea
    780.60 - Fever, Unspecified
    V69.2 - High Risk Sexual Behavior
    V72.62 - Laboratory Examination, General Medical Examination
    790.6 - Abnormal Blood Chemistry
    595.0 - Acute Cystitis
    257.2 - Testicular Hypofunction
    244.8 - Acquired Hypothyroidism
    616.10 - Vaginitis and Vulvitis, Unspecified
    V20.2 - Routine Child Health Examination
    786.2 - Cough
    790.29 - Other Abnormal Glucose
    530.81 - Esophageal Reflux
    V73.88 - Screening for Chlamydial Disease
    782.3 - Edema
    V77.91 - Screening for Lipoid Disorders
    780.4 - Dizziness and Giddiness
    V72.31 - Routine Gynecological Examination
    V22.1 - Supervision of Other Normal Pregnancy
    280.9 - Iron Deficiency Anemia, Unspecified
    300.00 - Anxiety State, Unspecified
    682.9 - Cellulitis and Abscess, Unspecified
    789.07 - Abdominal Pain, Generalized
    719.40 - Pain in Joint, Unspecified
    788.41 - Urinary Frequency
    784.0 - Headache
    783.1 - Abnormal Weight Gain
"""
import re

regex_code_desc = re.compile(r'(\b[V\d.]+)\s+-\s+(.*)')

code_descs = regex_code_desc.findall(candidate_list)


In [5]:
code_descs_dict = {code:desc for code, desc in code_descs}
code_descs_dict

{'V70.0': 'Routine Medical Exam',
 'V76.2': 'Screening for Malignant Neoplasm of Cervix',
 '401.1': 'Benign Hypertension',
 'V76.44': 'Screening for Malignant Neoplasm of Prostate',
 '250.00': 'Diabetes Mellitus without Complications, Type II or Unspecified Type',
 '401.9': 'Hypertension, Unspecified',
 '780.79': 'Other Malaise and Fatigue',
 '599.0': 'Urinary Tract Infection, Site Not Specified',
 '272.4': 'Other and Unspecified Hyperlipidemia',
 'V72.60': 'Laboratory Examination, Unspecified',
 '244.9': 'Hypothyroidism, Unspecified',
 'V58.69': 'Long-term (current) Use of Other Medications',
 '788.1': 'Dysuria',
 '272.2': 'Mixed Hyperlipidemia',
 '272.0': 'Pure Hypercholesterolemia',
 '268.9': 'Unspecified Vitamin D Deficiency',
 '462': 'Acute Pharyngitis',
 '250.02': 'Diabetes Mellitus without Complications, Type II or Unspecified Type, Uncontrolled',
 'V58.61': 'Long-term (current) Use of Anticoagulants',
 '789.00': 'Abdominal Pain, Unspecified Site',
 '285.9': 'Anemia, Unspecified

In [6]:
silver_df_main_og = silver_df_main

In [7]:
import ast

sentence_list = []
for x in eval(silver_df_main_og.iloc[0, -1]):
    datas = ast.literal_eval(x)
    for data in datas:
        sentence_list.append(data)

for i, snt in enumerate(sentence_list):
    print(f'{i+1}.) ',snt[1])

1.)  427.31
2.)  427.31
3.)  427.31
4.)  250.02
5.)  250.02
6.)  272.0
7.)  599.70
8.)  599.70
9.)  682.9
10.)  287.9
11.)  250.00
12.)  250.02
13.)  427.31
14.)  427.31


In [8]:
# import re
# icd_regex = r'(?:\b\w+\b\s*)?\b\d{3,5}\.?\d{2,4}\b'
# code_list = []
# seen_codes = set()
# for x in eval(silver_df_main.iloc[0,-1]):
#     # code_list.append(re.findall(icd_regex, x))
#     codes = re.findall(icd_regex, x)
#     for code in codes:
#         if code not in seen_codes:
#             seen_codes.add(code)
#             code_list.append(code)
#     # print(x)

# print(code_list)


In [9]:
doc_list = silver_df_main['ICD9_CODE'].tolist()

In [10]:
from tqdm import tqdm
doc_sentences = []
doc_no = 0
doc_no_not_proccessed = []
for doc in tqdm(doc_list):
    try:
        sentence_list = []
        for x in eval(doc):
            datas = ast.literal_eval(x)
            for data in datas:
                sentence_list.append(data)
        
        if len(sentence_list) < 5: # If the sentence level evidences contain less than 3 sentences we will remove that document
            raise Exception("Too less sentences")
        doc_sentences.append(sentence_list)
        doc_no +=1
    except:
        doc_no_not_proccessed.append(doc_no)
        doc_no +=1

100%|██████████| 1450/1450 [00:00<00:00, 8007.94it/s]


In [11]:
len(doc_no_not_proccessed) # This documents have some sort of errors and have to be removed 

619

**Now we need to determine the maximum number of sentences we can keep for each document, for the sake of simplicity I'll take it to be the minimum number of evidence sentences in each Document**

In [12]:
len(doc_sentences[0])

14

In [13]:
min_sentences = 2**31

for no_sent in doc_sentences:
    if min_sentences > len(no_sent):
        min_sentences = len(no_sent)

print(min_sentences)


5


The Minimum number of sentences in a document is 3 so for all documents we'll keep the 1st 3 sentences only

In [14]:

xD = []
for index in silver_df_main.index:
    try:
        if index in doc_no_not_proccessed:
            silver_df_main = silver_df_main.drop(index, axis=0)
    except:
        xD.append(index)
        # print(f"Index that couldn't be dropped: {index}")
len(xD)

0

In [15]:
silver_df_main = silver_df_main.reset_index().drop('index', axis = 1)

In [16]:
silver_df_main

,SUBJECT_ID,HADM_ID,TEXT,ICD9_CODE
0,55677,108847.0,Admission Date: [**2134-4-5**] D...,"[""[('This 79 year old male who is known to car..."
1,55122,160937.0,Admission Date: [**2192-3-27**] ...,['[(\'Patient underwent uncomplicated penile i...
2,88157,168232.0,Admission Date: [**2190-2-23**] ...,"['[(""Patient is a 51 yo female with no cardiac..."
3,31290,159495.0,Admission Date: [**2201-10-14**] ...,"[""[\n('Patient is 58-year-old man with history..."
4,17341,151110.0,Admission Date: [**2132-6-9**] D...,"[""[\n ('The patient remained on the surgical ..."
...,...,...,...,...
826,93628,134497.0,Admission Date: [**2177-3-24**] ...,"[""[('Social History: Patient had been maried f..."
827,14667,172924.0,Admission Date: [**2138-8-7**] D...,"[""[('Pt has >100pk/yrs and continues to smoke'..."
828,25235,182008.0,Admission Date: [**2167-6-10**] ...,"['[\n (""52 yo M with PMH DM I, CAD s/p CABG..."
829,18736,152789.0,Admission Date: [**2109-8-30**] ...,"[""[\n ('Patient recorded as having No Known..."


In [17]:
len(doc_sentences)

831

In [18]:
silver_df_main['doc_sentences'] = pd.Series([x[:5] for x in doc_sentences], index = silver_df_main.index)

In [19]:
# silver_df_main['doc_sentences'] = silver_df_main['doc_sentences'][:5]

In [20]:
for x in silver_df_main['doc_sentences'].to_list():
    if len(x) != 5:
        raise Exception('Some sentences are not equal to 5')
print("We're Good to proceed")
    

We're Good to proceed


In [21]:
silver_df_main

,SUBJECT_ID,HADM_ID,TEXT,ICD9_CODE,doc_sentences
0,55677,108847.0,Admission Date: [**2134-4-5**] D...,"[""[('This 79 year old male who is known to car...",[(This 79 year old male who is known to cardia...
1,55122,160937.0,Admission Date: [**2192-3-27**] ...,['[(\'Patient underwent uncomplicated penile i...,[(Patient underwent uncomplicated penile impla...
2,88157,168232.0,Admission Date: [**2190-2-23**] ...,"['[(""Patient is a 51 yo female with no cardiac...",[(Patient is a 51 yo female with no cardiac hi...
3,31290,159495.0,Admission Date: [**2201-10-14**] ...,"[""[\n('Patient is 58-year-old man with history...",[(Patient is 58-year-old man with history of D...
4,17341,151110.0,Admission Date: [**2132-6-9**] D...,"[""[\n ('The patient remained on the surgical ...",[(The patient remained on the surgical service...
...,...,...,...,...,...
826,93628,134497.0,Admission Date: [**2177-3-24**] ...,"[""[('Social History: Patient had been maried f...",[(Social History: Patient had been maried for ...
827,14667,172924.0,Admission Date: [**2138-8-7**] D...,"[""[('Pt has >100pk/yrs and continues to smoke'...","[(Pt has >100pk/yrs and continues to smoke, 25..."
828,25235,182008.0,Admission Date: [**2167-6-10**] ...,"['[\n (""52 yo M with PMH DM I, CAD s/p CABG...","[(52 yo M with PMH DM I, CAD s/p CABG x2, asth..."
829,18736,152789.0,Admission Date: [**2109-8-30**] ...,"[""[\n ('Patient recorded as having No Known...",[(Patient recorded as having No Known Allergie...


### Load the Gold levels (To reduce complexity of processing we'll use the ICD Codes predicted by GPT-4)

In [22]:
gold_df = pd.read_excel('data/silver_labels.xlsx')


In [23]:
gold_df = gold_df.iloc[:1450, :]
gold_df

,SUBJECT_ID,HADM_ID,TEXT,ICD9_CODE,encoded_ICD9_CODES
0,55677,108847,Admission Date: [**2134-4-5**] D...,"['427.31', 'V58.61', '401.9', '272.0', '272.2'...",[0 0 0 ... 0 0 0]
1,55122,160937,Admission Date: [**2192-3-27**] ...,"['996.1', '996.42', '414.00', 'V45.81', 'V58.6...",[0 0 0 ... 0 0 0]
2,11892,193055,Admission Date: [**2152-6-26**] Dischar...,"['401.9', '250.00', '250.02', '427.31', '427.3...",[0 0 0 ... 0 0 0]
3,13913,153057,Admission Date: [**2152-3-9**] Discharg...,"['578.9', '414.00', '745.5', '428.0', '250.00'...",[0 0 0 ... 0 0 0]
4,21553,188632,Admission Date: [**2184-12-24**] ...,"['276.7', '276.1', '458.0', 'E878.2', 'E878.7']",[0 0 0 ... 0 0 0]
...,...,...,...,...,...
1445,70182,182356,Admission Date: [**2110-2-9**] D...,"['426.4', '394.1', '396.3', '429.3', '38.85', ...",[0 0 0 ... 0 0 0]
1446,25235,182008,Admission Date: [**2167-6-10**] ...,"['190.0', '198.5', '416.0', '491.21', '518.4',...",[0 0 0 ... 0 0 0]
1447,18736,152789,Admission Date: [**2109-8-30**] ...,"['427.31', '599.0', '276.9', '518.82', '428.0'...",[0 0 0 ... 0 0 0]
1448,42757,167098,Admission Date: [**2154-10-6**] ...,"['349.82', '276.2', '458.29', '038.9', '427.31...",[0 0 0 ... 0 0 0]


In [24]:
for index in gold_df.index:
    if index in doc_no_not_proccessed:
        gold_df = gold_df.drop(index, axis=0)

gold_df = gold_df.reset_index().drop('index', axis=1)
gold_df

,SUBJECT_ID,HADM_ID,TEXT,ICD9_CODE,encoded_ICD9_CODES
0,55677,108847,Admission Date: [**2134-4-5**] D...,"['427.31', 'V58.61', '401.9', '272.0', '272.2'...",[0 0 0 ... 0 0 0]
1,55122,160937,Admission Date: [**2192-3-27**] ...,"['996.1', '996.42', '414.00', 'V45.81', 'V58.6...",[0 0 0 ... 0 0 0]
2,88157,168232,Admission Date: [**2190-2-23**] ...,"['424.0', '396.3', '493.90', '491.21', '424.0'...",[0 0 0 ... 0 0 0]
3,31290,159495,Admission Date: [**2201-10-14**] ...,"['427.31', '250.02', '780.60', 'V58.61', '599....",[0 0 0 ... 0 0 0]
4,17341,151110,Admission Date: [**2132-6-9**] D...,"['01.04', '32.0', '41.2', '34.92', '38.09', '8...",[0 0 0 ... 0 0 0]
...,...,...,...,...,...
826,93628,134497,Admission Date: [**2177-3-24**] ...,"['800.00', '850.9', '305.00', '303.90', 'V87.3...",[0 0 0 ... 0 0 0]
827,14667,172924,Admission Date: [**2138-8-7**] D...,"['491.21', '250.00', '244.9', '305.1', '276.8'...",[0 0 0 ... 0 0 0]
828,25235,182008,Admission Date: [**2167-6-10**] ...,"['190.0', '198.5', '416.0', '491.21', '518.4',...",[0 0 0 ... 0 0 0]
829,18736,152789,Admission Date: [**2109-8-30**] ...,"['427.31', '599.0', '276.9', '518.82', '428.0'...",[0 0 0 ... 0 0 0]


In [25]:
gold_df['ICD9_CODE'] = pd.Series([eval(x) for x in gold_df['ICD9_CODE']], index=gold_df.index)

In [26]:
gold_df

,SUBJECT_ID,HADM_ID,TEXT,ICD9_CODE,encoded_ICD9_CODES
0,55677,108847,Admission Date: [**2134-4-5**] D...,"[427.31, V58.61, 401.9, 272.0, 272.2, 244.9, V...",[0 0 0 ... 0 0 0]
1,55122,160937,Admission Date: [**2192-3-27**] ...,"[996.1, 996.42, 414.00, V45.81, V58.61, 443.9,...",[0 0 0 ... 0 0 0]
2,88157,168232,Admission Date: [**2190-2-23**] ...,"[424.0, 396.3, 493.90, 491.21, 424.0, 795.00, ...",[0 0 0 ... 0 0 0]
3,31290,159495,Admission Date: [**2201-10-14**] ...,"[427.31, 250.02, 780.60, V58.61, 599.0, 682.9,...",[0 0 0 ... 0 0 0]
4,17341,151110,Admission Date: [**2132-6-9**] D...,"[01.04, 32.0, 41.2, 34.92, 38.09, 87.66, 22.56...",[0 0 0 ... 0 0 0]
...,...,...,...,...,...
826,93628,134497,Admission Date: [**2177-3-24**] ...,"[800.00, 850.9, 305.00, 303.90, V87.39, 348.30...",[0 0 0 ... 0 0 0]
827,14667,172924,Admission Date: [**2138-8-7**] D...,"[491.21, 250.00, 244.9, 305.1, 276.8, 293.0, 2...",[0 0 0 ... 0 0 0]
828,25235,182008,Admission Date: [**2167-6-10**] ...,"[190.0, 198.5, 416.0, 491.21, 518.4, 785.52, V...",[0 0 0 ... 0 0 0]
829,18736,152789,Admission Date: [**2109-8-30**] ...,"[427.31, 599.0, 276.9, 518.82, 428.0, 161.9, 2...",[0 0 0 ... 0 0 0]


In [27]:
silver_df_main

,SUBJECT_ID,HADM_ID,TEXT,ICD9_CODE,doc_sentences
0,55677,108847.0,Admission Date: [**2134-4-5**] D...,"[""[('This 79 year old male who is known to car...",[(This 79 year old male who is known to cardia...
1,55122,160937.0,Admission Date: [**2192-3-27**] ...,['[(\'Patient underwent uncomplicated penile i...,[(Patient underwent uncomplicated penile impla...
2,88157,168232.0,Admission Date: [**2190-2-23**] ...,"['[(""Patient is a 51 yo female with no cardiac...",[(Patient is a 51 yo female with no cardiac hi...
3,31290,159495.0,Admission Date: [**2201-10-14**] ...,"[""[\n('Patient is 58-year-old man with history...",[(Patient is 58-year-old man with history of D...
4,17341,151110.0,Admission Date: [**2132-6-9**] D...,"[""[\n ('The patient remained on the surgical ...",[(The patient remained on the surgical service...
...,...,...,...,...,...
826,93628,134497.0,Admission Date: [**2177-3-24**] ...,"[""[('Social History: Patient had been maried f...",[(Social History: Patient had been maried for ...
827,14667,172924.0,Admission Date: [**2138-8-7**] D...,"[""[('Pt has >100pk/yrs and continues to smoke'...","[(Pt has >100pk/yrs and continues to smoke, 25..."
828,25235,182008.0,Admission Date: [**2167-6-10**] ...,"['[\n (""52 yo M with PMH DM I, CAD s/p CABG...","[(52 yo M with PMH DM I, CAD s/p CABG x2, asth..."
829,18736,152789.0,Admission Date: [**2109-8-30**] ...,"[""[\n ('Patient recorded as having No Known...",[(Patient recorded as having No Known Allergie...


In [28]:
silver_df_main.iloc[0,-1][1][1]

'427.31'

In [29]:
from typing import Dict, List
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd

class Prepare_Tabulated_Data():
    def __init__(self, code_description: Dict) -> None:
        self.all_labels = list(code_description.keys())[:20]
        self.all_descriptions = list(code_description.values())[:20]
        self.mlb = MultiLabelBinarizer(classes= self.all_labels, sparse_output=False)
    

    
    def fit(self, labels)->None:
        if isinstance(labels, list) and all(isinstance(label, list) for label in labels):
            self.mlb.fit(labels)
        else:
            raise ValueError("Labels should be a list of lists.")
        

    def transform(self, labels)->None:
        if isinstance(labels, list) and all(isinstance(label, list) for label in labels):

            encoded_labels = self.mlb.transform(labels)
            return encoded_labels
        else:
            raise ValueError("Labels should be a list of lists.")   



    def perpare_labels(self, gold_row, silver_row) -> pd.DataFrame:
        gold_encoding = self.transform(labels=[gold_row.ICD9_CODE])


        sentence_x = []
        Silver_vector = []
        Gold_vecor = []
        ICD_Desc = []
        for sentence_and_code in silver_row.doc_sentences:
            
            sentence = sentence_and_code[0]
            silver_code = sentence_and_code[1]

            silver_encoding = self.transform([[silver_code]])

            sentence_x.append(sentence)
            Silver_vector.append(silver_encoding)
            Gold_vecor.append(gold_encoding)
            ICD_Desc.append(self.all_descriptions)
        
        dicti = {'Sentence': sentence_x,
                 'Silver_Labels': Silver_vector,
                 'Gold_Labels': Gold_vecor,
                 'ICD_Desc': ICD_Desc}
        
        df = pd.DataFrame(dicti)

        return df
    
    def Tabulate_Data(self,
                        silver_df: pd.DataFrame,
                        gold_df: pd.DataFrame)-> List:
        
        self.fit([self.all_labels])
        final_ans = []
        for index in silver_df.index:
            silver_row = silver_df.iloc[index]
            gold_row = gold_df.iloc[index]

            label_df = self.perpare_labels(gold_row=gold_row, silver_row=silver_row)
            final_ans.append(label_df)
        
        return final_ans

In [30]:
encoder = Prepare_Tabulated_Data(code_description=code_descs_dict)
Tabulated_Matrix = encoder.Tabulate_Data(silver_df=silver_df_main, gold_df=gold_df)

c:\Users\singh\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:895: UserWarning: unknown class(es) ['414.00', '427.31', '428.0', '443.9', '996.62', 'V77.91'] will be ignored
  warnings.warn(
c:\Users\singh\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:895: UserWarning: unknown class(es) ['427.31'] will be ignored
  warnings.warn(
c:\Users\singh\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:895: UserWarning: unknown class(es) ['285.9', '414.00', '427.31', '440.0', '443.89', '443.9', '530.81', '573.3', '682.9', '780.4', '788.41', '996.1', '996.42', 'V10.3', 'V13.09', 'V45.81', 'V45.89', 'V64.41'] will be ignored
  warnings.warn(
c:\Users\singh\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:895: UserWarning: unknown class(es) ['786.2;V70.0'] will be ignored
  warnings.warn(
c:\Users\singh\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:895: UserWarning: unknown class(es) ['401.1;427.31;413.9;414.01;413.9'] will be ignored
  

In [31]:
Tabulated_Matrix[70]

,Sentence,Silver_Labels,Gold_Labels,ICD_Desc
0,The patient complained of left hip pain.,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,...","[Routine Medical Exam, Screening for Malignant..."
1,Left femoral fracture,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,...","[Routine Medical Exam, Screening for Malignant..."
2,Left femoral fracture extending from the femor...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,...","[Routine Medical Exam, Screening for Malignant..."
3,Left hip pain,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,...","[Routine Medical Exam, Screening for Malignant..."
4,Left femoral fracture extending from the femor...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,...","[Routine Medical Exam, Screening for Malignant..."


## Preparing the For Model

In [32]:
X = []

for matrix in Tabulated_Matrix:
    selected_columns = matrix.iloc[:, [0, -1]]
    X.append(selected_columns)

In [33]:
X[0]

,Sentence,ICD_Desc
0,This 79 year old male who is known to cardiac ...,"[Routine Medical Exam, Screening for Malignant..."
1,He was transfered to [**Hospital1 18**] for fu...,"[Routine Medical Exam, Screening for Malignant..."
2,IMPRESSION: Severe aortic valve stenosis.,"[Routine Medical Exam, Screening for Malignant..."
3,Novolog insulin 8 units [**Hospital1 **] (befo...,"[Routine Medical Exam, Screening for Malignant..."
4,Insulin lispro,"[Routine Medical Exam, Screening for Malignant..."


In [34]:
import torch
from torch.utils.data import DataLoader, Dataset

BATCH_SIZE = 4


# GPU loading
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Prepare Custom Dataset
class Text_Dataset(Dataset):
    def __init__(self, texts):
        self.text = texts
    
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, index):
        return self.text[index]
    

Sentences = [df.iloc[:,0].to_list() for df in Tabulated_Matrix]
ICD_Descriptions = [df.iloc[:,-1].to_list() for df in Tabulated_Matrix]
silver_labels = [df.iloc[:,1].to_list() for df in Tabulated_Matrix]
gold_labels = [df.iloc[:,2].to_list() for df in Tabulated_Matrix]

# Flattening the inputs
Sentences_flattened = [item for sublist in Sentences for item in sublist]
ICD_Descriptions_flattened = [item for sublist in ICD_Descriptions for item in sublist]

# Silver flattened

# Gold flattened


sentence_data = Text_Dataset(Sentences_flattened)
ICD_Descriptions_data = Text_Dataset(ICD_Descriptions_flattened)

sentence_loader = DataLoader(sentence_data, batch_size=BATCH_SIZE, shuffle=False)
ICD_Descriptions_loader = DataLoader(ICD_Descriptions_data, batch_size=BATCH_SIZE, shuffle=False)



## Prepare the Model

In [35]:
from torch import nn
import torch.nn.functional as F
from transformers import DistilBertTokenizer, DistilBertModel

# GPU loading
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Verifier_Model(nn.Module):
    def __init__(self, hidden_units: int = 256, output_shape: int = 20, input_shape: int = 768):
        super().__init__()
        self.tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
        self.TextEncoderModel = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.MLP = nn.Sequential(nn.Linear(in_features=input_shape, out_features=hidden_units),
                                 nn.ReLU(),
                                 nn.Linear(in_features=hidden_units, out_features=output_shape),
                                 )
    
    def TextEncoder(self, text):
        model = self.TextEncoderModel
        tokenizer = self.tokenizer

        model.eval()

        with torch.no_grad():
            inputs = tokenizer(text, padding=True, truncation=True, return_tensors='pt').to(device)
            outputs = model(**inputs)

            latent_rep = outputs.last_hidden_state.mean(dim=1)
        
        return latent_rep
    
    def CalculateSimilarity(self, rep1, rep2):

        rep1 = F.normalize(rep1, p=2, dim=1)
        rep2 = F.normalize(rep2, p=2, dim=1)
        similarity = F.cosine_similarity(rep1, rep2, dim=1)
        return similarity
    
    def forward(self, text1, text2):
        rep1 = self.TextEncoder([text1]).to(device)  # Shape: [1, 768]
        rep2 = self.TextEncoder(text2).to(device)    # Shape: [20, 768]

        similarity = self.CalculateSimilarity(rep1, rep2)  # Shape: [1, 20]

        similarity_expanded = similarity.unsqueeze(-1)  # Shape: [1, 20, 1]
        rep1_expanded = rep1.unsqueeze(1)  # Shape: [1, 1, 768]
        weighted_rep1 = rep1_expanded * similarity_expanded  # Shape: [1, 20, 768]

        hf = rep2 + weighted_rep1.squeeze(0)  # Shape: [20, 768]

        output = self.MLP(hf)  # Shape: [20, 20]

        return output.mean(dim=0).unsqueeze(0)  # Shape: [1, 20]


        # return output


In [36]:
test_s = Sentences_flattened[0]
text_i = (ICD_Descriptions_flattened[0])

In [37]:
model = Verifier_Model()
model.to(device)
ans = model(test_s, text_i)

c:\Users\singh\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [38]:
ans.shape

torch.Size([1, 20])

In [39]:
input1 = torch.randn(100, 128)
input2 = torch.randn(100, 128)
output = F.cosine_similarity(input1, input2)
print(output.shape)

torch.Size([100])


In [40]:

train_features = next(iter(sentence_loader))

In [41]:
train_features[0]

'This 79 year old male who is known to cardiac surgery with critical aortic stenosis, having refused surgical intervention in the past, s/p valuloplasy x2 [**2132-10-15**]/[**2133-12-15**], who was transferred from an OSH for acute chest pain with troponin bump to 4.4.'

In [42]:
print("Number of classes:", len(Tabulated_Matrix[0]['Silver_Labels'][0]))
print("Type of Silver_Labels:", type(Tabulated_Matrix[0]['Silver_Labels']))
print("Type of Gold_Labels:", type(Tabulated_Matrix[0]['Gold_Labels']))
print("Number of unique ICD codes:", len(encoder.all_labels))

Number of classes: 1
Type of Silver_Labels: <class 'pandas.core.series.Series'>
Type of Gold_Labels: <class 'pandas.core.series.Series'>
Number of unique ICD codes: 20


In [43]:
def compute_loss(logits, gold_labels, silver_labels):
    """
    Compute the custom loss function combining gold and silver label losses.
    
    :param logits: Tensor of shape (batch_size, num_classes)
    :param gold_labels: Tensor of shape (batch_size, num_classes)
    :param silver_labels: Tensor of shape (batch_size, num_sentences, num_classes)
    :return: Combined loss
    """
    # Compute the gold label loss (binary cross-entropy)
    gold_loss = F.binary_cross_entropy_with_logits(logits, gold_labels, reduction='sum')
    
    # Compute the silver label loss (binary cross-entropy)
    silver_loss = F.binary_cross_entropy_with_logits(logits.unsqueeze(1).expand_as(silver_labels), silver_labels, reduction='sum')
    
    # Total loss
    total_loss = gold_loss + silver_loss
    return total_loss

In [44]:
from torch import optim
import numpy as np
model = Verifier_Model(output_shape=20).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-5)

num_epochs = 10
for epoch in range(num_epochs):
    total_loss = 0
    for batch_sentences, batch_icd_descriptions in zip(sentence_loader, ICD_Descriptions_loader):
        optimizer.zero_grad()
        
        batch_size = len(batch_sentences)
        
        # Forward pass
        logits = torch.cat([model(sentence, icd_descriptions) for sentence, icd_descriptions in zip(batch_sentences, batch_icd_descriptions)], dim=0)
        
        # Prepare gold and silver labels
        gold_labels = torch.tensor(np.vstack([x.to_numpy() for x in batch_sentences['Gold_Labels']])).float().to(device)
        silver_labels = torch.tensor(np.array([x.to_numpy() for x in batch_sentences['Silver_Labels']])).float().to(device)
        
        # Compute loss
        loss = compute_loss(logits, gold_labels, silver_labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss}")

TypeError: list indices must be integers or slices, not str

In [45]:
# Get a sample batch
sample_sentences = next(iter(sentence_loader))
sample_icd_descriptions = next(iter(ICD_Descriptions_loader))

# Forward pass
logits = torch.cat([model(sentence, icd_descriptions) for sentence, icd_descriptions in zip(sample_sentences, sample_icd_descriptions)], dim=0)

# Prepare gold and silver labels
gold_labels = torch.tensor(np.vstack([x.to_numpy() for x in sample_sentences['Gold_Labels']])).float().to(device)
silver_labels = torch.tensor(np.array([x.to_numpy() for x in sample_sentences['Silver_Labels']])).float().to(device)

# Compute loss
loss = compute_loss(logits, gold_labels, silver_labels)

print("Pipeline test loss:", loss.item())

TypeError: list indices must be integers or slices, not str

New model and dataloader testing

In [46]:
import torch
from torch.utils.data import DataLoader, Dataset

BATCH_SIZE = 4
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MedicalDataset(Dataset):
    def __init__(self, tabulated_matrix):
        self.data = tabulated_matrix
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        item = self.data[index]
        return {
            'sentences': item['Sentence'].tolist(),
            'icd_descriptions': item['ICD_Desc'][0],
            'silver_labels': torch.tensor([sl[0] for sl in item['Silver_Labels']], dtype=torch.float),
            'gold_labels': torch.tensor(item['Gold_Labels'][0][0], dtype=torch.float)  # Remove extra dimension
        }

def collate_fn(batch):
    return {
        'sentences': [item['sentences'] for item in batch],
        'icd_descriptions': [item['icd_descriptions'] for item in batch],
        'silver_labels': torch.stack([item['silver_labels'] for item in batch]),
        'gold_labels': torch.stack([item['gold_labels'] for item in batch])
    }

# Create dataset and dataloader
dataset = MedicalDataset(Tabulated_Matrix)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

In [47]:
# Get a batch from your data loader
sample_batch = next(iter(dataloader))

print("Batch size:", len(sample_batch['sentences']))
print("Sample sentences:", len(sample_batch['sentences'][0]))
print("Sample ICD descriptions length:", len(sample_batch['icd_descriptions'][0]))
print("Silver labels shape:", sample_batch['silver_labels'].shape)
print("Gold labels shape:", sample_batch['gold_labels'].shape)
print("Silver labels sample:", sample_batch['silver_labels'][0][0])  # First sentence, first document
print("Gold labels sample:", sample_batch['gold_labels'][0])  # First document

Batch size: 4
Sample sentences: 5
Sample ICD descriptions length: 20
Silver labels shape: torch.Size([4, 5, 20])
Gold labels shape: torch.Size([4, 20])
Silver labels sample: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Gold labels sample: tensor([0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1.,
        1., 0.])


C:\Users\singh\AppData\Local\Temp\ipykernel_16896\1813202218.py:19: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:277.)
  'silver_labels': torch.tensor([sl[0] for sl in item['Silver_Labels']], dtype=torch.float),


In [55]:
import torch
import torch.nn as nn
import torch.optim as optim

class Verifier_Model(nn.Module):
    def __init__(self, hidden_units: int = 256, output_shape: int = 20, input_shape: int = 768):
        super().__init__()
        self.tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
        self.TextEncoderModel = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.model = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.MLP = nn.Sequential(
            nn.Linear(in_features=input_shape, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=output_shape),
        )
    
    def TextEncoder(self, text):
        if isinstance(text, list) and isinstance(text[0], str):
            inputs = self.tokenizer(text, padding=True, truncation=True, return_tensors='pt').to(device)
        elif isinstance(text, str):
            inputs = self.tokenizer([text], padding=True, truncation=True, return_tensors='pt').to(device)
        else:
            raise ValueError("Input must be either a string or a list of strings")
        
        outputs = self.TextEncoderModel(**inputs)
        return outputs.last_hidden_state.mean(dim=1)
    
    def CalculateSimilarity(self, rep1, rep2):
        rep1 = F.normalize(rep1, p=2, dim=1)
        rep2 = F.normalize(rep2, p=2, dim=1)
        return F.cosine_similarity(rep1.unsqueeze(1), rep2.unsqueeze(0), dim=2)
    
    def forward(self, sentences, icd_descriptions):
        batch_size = len(sentences)
        all_outputs = []
        
        for i in range(batch_size):
            doc_sentences = sentences[i]
            doc_icd_desc = icd_descriptions[i]
            
            rep1 = self.TextEncoder(doc_sentences)  # Shape: [num_sentences, 768]
            rep2 = self.TextEncoder(doc_icd_desc)  # Shape: [20, 768]
            
            similarity = self.CalculateSimilarity(rep1, rep2)  # Shape: [num_sentences, 20]
            
            similarity_expanded = similarity.unsqueeze(-1)  # Shape: [num_sentences, 20, 1]
            rep1_expanded = rep1.unsqueeze(1)  # Shape: [num_sentences, 1, 768]
            weighted_rep1 = rep1_expanded * similarity_expanded  # Shape: [num_sentences, 20, 768]
            
            hf = rep2.unsqueeze(0) + weighted_rep1  # Shape: [num_sentences, 20, 768]
            
            output = self.MLP(hf.view(-1, 768)).view(len(doc_sentences), 20, -1)  # Shape: [num_sentences, 20, output_shape]
            sentence_outputs = output.mean(dim=0)  # Shape: [20, output_shape]
            
            all_outputs.append(sentence_outputs)
        
        return torch.stack(all_outputs)  # Shape: [batch_size, 20, output_shape]

# Custom loss function
def compute_loss(logits, gold_labels, silver_labels):
    #print("Logits shape:", logits.shape)
    #print("Gold labels shape:", gold_labels.shape)
    #print("Silver labels shape:", silver_labels.shape)
    
    batch_size, num_icd, num_classes = logits.shape
    num_sentences = silver_labels.size(1)
    
    # For gold loss, we need to aggregate the logits across the ICD dimension
    logits_for_gold = logits.mean(dim=1)  # Shape: [batch_size, num_classes]
    
    # Compute the gold label loss (binary cross-entropy)
    gold_loss = F.binary_cross_entropy_with_logits(
        logits_for_gold, gold_labels, reduction='sum'
    )
    
    # Expand logits to match silver_labels shape
    expanded_logits = logits.unsqueeze(1).expand(-1, num_sentences, -1, -1)
    
    # Reshape silver_labels to match expanded_logits
    silver_labels_expanded = silver_labels.unsqueeze(2).expand(-1, -1, num_icd, -1)
    
    # Compute the silver label loss (binary cross-entropy)
    silver_loss = F.binary_cross_entropy_with_logits(
        expanded_logits, silver_labels_expanded, reduction='sum'
    )
    
    # Total loss
    total_loss = gold_loss + silver_loss
    return total_loss

# Training loop
model = Verifier_Model().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-5)

num_epochs = 10
for epoch in range(num_epochs):
    total_loss = 0
    for batch in dataloader:
        sentences = batch['sentences']
        icd_descriptions = batch['icd_descriptions']
        silver_labels = batch['silver_labels'].to(device)
        gold_labels = batch['gold_labels'].to(device)

        optimizer.zero_grad()
        
        logits = model(sentences, icd_descriptions)
        #print("Output logits shape:", logits.shape)
        
        loss = compute_loss(logits, gold_labels, silver_labels)
        
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss}")

Sentences type: <class 'list'>, Content: [['This 79 year old male who is known to cardiac surgery with critical aortic stenosis, having refused surgical intervention in the past, s/p valuloplasy x2 [**2132-10-15**]/[**2133-12-15**], who was transferred from an OSH for acute chest pain with troponin bump to 4.4.', 'He was transfered to [**Hospital1 18**] for further mangangment of aortic stenosis.', 'IMPRESSION: Severe aortic valve stenosis.', 'Novolog insulin 8 units [**Hospital1 **] (before breakfast and before\nsupper)', 'Insulin lispro'], ['Patient underwent uncomplicated penile implant removal and replacement with urology on [**3-27**].', 'He reported that his chest was "on fire", and has never had such severe symptoms before.', 'He underwent immediate cardiac cath which showed 3VD with 70% mid LAD, 90% D2, 80% OM, 70% r-PDA, and 90% RCA lesion.', 'The patient was brought to the Operating Room on [**3-29**] and underwent revascularization(LIMA-LAD, SVG-RCA and PDA, SVG-OM).', "The 

RuntimeError: The size of tensor a (87) must match the size of tensor b (19) at non-singleton dimension 2

In [52]:
logits.shape

torch.Size([4, 20, 20])

In [57]:
# Assume num_epochs and the model setup code is already provided as in previous examples

num_epochs = 10
for epoch in range(num_epochs):
    print(f"Starting Epoch {epoch+1}/{num_epochs}")
    for batch_sentences, batch_icd_descriptions in zip(sentence_loader, ICD_Descriptions_loader):
        # Print the structure and content of each batch
        print("\nBatch Sentences:")
        print(batch_sentences)
        print("Type of each element in Batch Sentences:", type(batch_sentences[0]))
        
        print("\nBatch ICD Descriptions:")
        print(batch_icd_descriptions)
        print("Type of each element in Batch ICD Descriptions:", type(batch_icd_descriptions[0]))
        
        # If labels are part of sentences or descriptions, print them specifically
        if isinstance(batch_sentences, list) and isinstance(batch_sentences[0], dict):
            if 'Gold_Labels' in batch_sentences[0]:
                print("\nGold Labels in Batch:")
                print([x['Gold_Labels'] for x in batch_sentences])
            if 'Silver_Labels' in batch_sentences[0]:
                print("\nSilver Labels in Batch:")
                print([x['Silver_Labels'] for x in batch_sentences])
        
        # If labels are loaded separately and you know the loader or method, print them
        # For example, if you have a function or loader like `label_loader` (this is hypothetical):
        # gold_labels, silver_labels = next(label_loader)
        # print("\nGold Labels:", gold_labels)
        # print("Silver Labels:", silver_labels)
        
        # Here you could add a break to stop after printing the first batch of data
        break
    
    # Break after the first epoch to prevent too much data from printing at once
    break


Starting Epoch 1/10

Batch Sentences:
['This 79 year old male who is known to cardiac surgery with critical aortic stenosis, having refused surgical intervention in the past, s/p valuloplasy x2 [**2132-10-15**]/[**2133-12-15**], who was transferred from an OSH for acute chest pain with troponin bump to 4.4.', 'He was transfered to [**Hospital1 18**] for further mangangment of aortic stenosis.', 'IMPRESSION: Severe aortic valve stenosis.', 'Novolog insulin 8 units [**Hospital1 **] (before breakfast and before\nsupper)']
Type of each element in Batch Sentences: <class 'str'>

Batch ICD Descriptions:
[('Routine Medical Exam', 'Routine Medical Exam', 'Routine Medical Exam', 'Routine Medical Exam'), ('Screening for Malignant Neoplasm of Cervix', 'Screening for Malignant Neoplasm of Cervix', 'Screening for Malignant Neoplasm of Cervix', 'Screening for Malignant Neoplasm of Cervix'), ('Benign Hypertension', 'Benign Hypertension', 'Benign Hypertension', 'Benign Hypertension'), ('Screening for 